# CSI 4142 Data Science 
## Assignment 2 - Data Cleaning

### Identification

Name: Eli Wynn<br/>
Student Number: 300248135

Name: Jack Snelgrove<br/>
Student Number: 300247435


Our datasets have been uploaded from the public repository:

- [github.com/eli-wynn/Datasets](https://github.com/eli-wynn/Datasets)

Imports:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import stats

### Importing Datasets

In [ ]:
netflix  = "https://raw.githubusercontent.com/eli-wynn/Datasets/refs/heads/main/netflix_titles.csv"
netflixData = pd.read_csv(netflix)
startup = "https://raw.githubusercontent.com/eli-wynn/Datasets/refs/heads/main/startup.csv"
startupData = pd.read_csv(startup)

### Clean Data Checker

#### Data Type Error

A data type error occurs when the the data entered into a column doesnt match the data type assigned to that column. 


#### Parameters

#### Checker Code

#### Findings

#### Range Errors



#### Parameters

#### Checker Code

#### Findings

#### Format Errors

#### Parameters

#### Checker Code

#### Findings

#### Consistency Errors

#### Parameters

#### Checker Code

#### Findings

#### Uniqueness Errors

#### Parameters

#### Checker Code

#### Findings

#### Presence Errors

#### Parameters

#### Checker Code

#### Findings

#### Length Errors

#### Parameters

#### Checker Code

#### Findings

#### Lookup Errors

#### Parameters

#### Checker Code

#### Findings

#### Exact Duplicate Errors

#### Parameters

#### Checker Code

#### Findings

#### Near Duplicate Errors

#### Parameters

#### Checker Code

#### Findings

### Imputations

#### Test #1

a) Column: Funding Rounds - Numerical<br>


b) Remove 10% of cells in column 'Funding Rounds' - MCAR

We define y_true, which saves the values we remove so that we can evaluate the imputation method in part d

In [ ]:
startupDataMissingValues = startupData.copy()

num_rows = startupDataMissingValues.shape[0]
num_missing = int(num_rows * 0.1)  

missing_indices = np.random.choice(startupDataMissingValues.index, size=num_missing, replace=False)

y_true = startupDataMissingValues.loc[missing_indices, 'Funding Rounds']

startupDataMissingValues.loc[missing_indices, 'Funding Rounds'] = np.nan


c) Method 3: Random Sample Imputation -Univariate

In [ ]:
observed_values = startupDataMissingValues['Funding Rounds'].dropna()

def randomSampleImputation(x):
    if pd.isnull(x):
        return np.random.choice(observed_values)
    else:
        return x

startupDataMissingValues['Funding Rounds'] = startupDataMissingValues['Funding Rounds'].apply(randomSampleImputation)

d) Evaluate approach using Mean Squared Error and Root Mean Squared Error

In [ ]:
y_pred = startupDataMissingValues.loc[missing_indices, 'Funding Rounds']
mse = np.mean((y_true - y_pred)**2)
print('Column Minimum:', startupData['Funding Rounds'].min())
print('Column Maximum:', startupData['Funding Rounds'].max())
print("MSE:", mse)
rmse = np.sqrt(mse)
print("RMSE:", rmse)

After applying random sample imputation to the 'Funding Rounds' column, we evaluated the method using the Mean Squared Error (MSE) and Root Mean Squared Error (RMSE). The MSE was calculated to be ~ 15-18, which corresponds to an RMSE of approximately 4 rounds. Considering that the 'Funding Rounds' values range from 1 to 10, this indicates that, on average, our imputed values deviate from the true values by about 4 rounds—nearly 40% of the total range. This substantial error suggests that while random sample imputation may preserve the overall distribution of the observed data, it introduces a significant level of discrepancy, warranting further investigation into alternative imputation strategies.

#### Test #2 

a) Column: Investment Amount

b) If a row has a 'Funding Round' value of 1, it has a 75% chance of having an empty 'Ivestment Amount (USD)' cell, if 'Funding Round' = 2, it has a 30% chance of leaving 'Investment Amount (USD)' blank, and if 'Funding Round = 3, it has a 5% chance of leaving 'Investment Amount (USD) blank. - MAR

Assumption: Startups in early funding rounds might not want to disclode their investment amount yet.

We define y_true, which saves the values we remove so that we can evaluate the imputation method in part d

In [272]:
startupDataMissingValues2 = startupData.copy()

#assigns a random number to each row between 0-1
random_numbers = np.random.rand(startupDataMissingValues2.shape[0])

prob_1 = 0.75
prob_2 = 0.30
prob_3 = 0.05

shouldRemove = (
    ((startupDataMissingValues2['Funding Rounds'] == 1) & (random_numbers < prob_1)) |
    ((startupDataMissingValues2['Funding Rounds'] == 2) & (random_numbers < prob_2)) |
    ((startupDataMissingValues2['Funding Rounds'] == 3) & (random_numbers < prob_3))
)


y_true = startupDataMissingValues2.loc[shouldRemove, 'Investment Amount (USD)']

startupDataMissingValues2.loc[shouldRemove, 'Investment Amount (USD)'] = np.nan



c) Method 7: Regression Imputation / Predictive Imputation

In [273]:
# Create a copy for regression imputation
from sklearn.linear_model import LinearRegression

startupDataMissingValues2['Log Investment Amount'] = np.log(startupDataMissingValues2['Investment Amount (USD)'])


# 1. Split the data into training (non-missing) and missing groups
train_data = startupDataMissingValues2[startupDataMissingValues2['Log Investment Amount'].notnull()]
pred_data = startupDataMissingValues2[startupDataMissingValues2['Log Investment Amount'].isnull()]


# 2. Use a strongly correlated variable as predictor (e.g., 'Funding Rounds')
X_train = train_data[[
    'Funding Rounds',
    'Valuation (USD)', 
    'Number of Investors', 
    'Year Founded',
    'Growth Rate (%)'
]]
y_train = train_data['Log Investment Amount']

model = LinearRegression()
model.fit(X_train, y_train)

X_pred = pred_data[[
    'Funding Rounds',
    'Valuation (USD)', 
    'Number of Investors', 
    'Year Founded',
    'Growth Rate (%)'
]]

y_pred = model.predict(X_pred)

startupDataMissingValues2.loc[startupDataMissingValues2['Investment Amount (USD)'].isnull(), 'Investment Amount (USD)'] = y_pred

In [ ]:
y_pred = startupDataMissingValues2.loc[shouldRemove, 'Investment Amount (USD)']

print('Investment Amount Minimum:', startupData['Investment Amount (USD)'].min())
print('Investment Amount Maximum:', startupData['Investment Amount (USD)'].max())

mse = np.mean((y_true - y_pred)**2)
print("MSE:", mse)
rmse = np.sqrt(mse)
print("RMSE:", rmse)

Column Minimum: 1102610.0
Column Maximum: 4999543707.18
MSE: 7.501431958738974e+18
RMSE: 2738874213.748958


After applying regression/predictive imputation to the 'Investment Amount (USD)' column, we evaluated the method using the Mean Squared Error (MSE) and Root Mean Squared Error (RMSE). The MSE was calculated to be ~ 7 Quintilion - 8 Quintilion which corresponds to an RMSE of approximately 2.7 Billion - 2.9 Billion dollars. Considering that the 'Investment Amount (USD)' values range from 1 Million to 5 Billion, this indicates that, on average, our imputed values deviate from the true values by about 2.8 Billion dollars —nearly 60% of the total range. This regression model preformed really poorly in predicting Investment Amount. This could be for numerous reasons but it is probably because investment amount isnt strongly correlated with other columns in the data frame. I would recommend simply removing the row from the set as predicting this value or using other imputation methods are bound to be inacurate due to the nature of the column in this domain. 